# Fakestore Sales Analysis (L2)

Notebook ini menganalisis dataset **L2** hasil pipeline (DAG 99) menggunakan file:
- `assets/sample_output/l2/cleaned_data.csv`
- `assets/sample_output/l2/dim_product.csv`
- Ringkasan Data Quality: `assets/sample_output/dq-reports/run_results.json`

**Tujuan:**
1. Menyajikan metrik ringkas (Revenue, Orders, Items, Unique Products, AOV).
2. Visualisasi: tren penjualan harian, kategori teratas, produk teratas, serta distribusi harga.
3. Menyertakan snapshot hasil Data Quality untuk konteks validitas data.

In [ ]:
# ==== Setup & Konfigurasi ====
import os, json
import pandas as pd
import matplotlib.pyplot as plt
from textwrap import dedent
from datetime import datetime
from IPython.display import display, Markdown

# --- Toggle sumber data ---
SOURCE = "csv"  # "csv" (recommended) | "db"

# --- Paths CSV (from DAG 99 exports) ---
BASE = r"/opt/project/assets/sample_output"
FACT_CSV = os.path.join(BASE, "l2", "cleaned_data.csv")
DIM_CSV  = os.path.join(BASE, "l2", "dim_product.csv")
RUN_RESULTS = os.path.join(BASE, "dq-reports", "run_results.json")

print("Using SOURCE:", SOURCE)
print("BASE:", BASE)

## 1) Load Data (CSV default)

In [ ]:
# ==== Load Data ====
if SOURCE == "csv":
    # parse kolom tanggal untuk analisis time-series
    fact = pd.read_csv(FACT_CSV, parse_dates=["transaction_date","load_ds"], low_memory=False)
    dim  = pd.read_csv(DIM_CSV)
else:
    # Opsional: DB mode (matikan jika tidak dibutuhkan)
    import psycopg2
    conn = psycopg2.connect(
        host="POSTGRES_HOST", port=5432, dbname="POSTGRES_DB",
        user="POSTGRES_USER", password="POSTGRES_PASSWORD"
    )
    fact = pd.read_sql_query("select * from l2.fact_sales", conn, parse_dates=["transaction_date","load_ds"])
    dim  = pd.read_sql_query("select * from l2.dim_product", conn)

print("fact shape:", fact.shape)
print("dim shape :", dim.shape)

# Safety: pastikan total_sales ada
if "total_sales" not in fact.columns:
    fact["total_sales"] = fact["quantity"] * fact["price"]

# Kolom minimal yang digunakan analisis
required_fact_cols = {"transaction_date","product_id","product_name","category","quantity","price","total_sales"}
missing = required_fact_cols.difference(set(fact.columns))
if missing:
    print("WARNING: Missing columns in fact:", sorted(missing))

# Tampilkan head untuk sanity check
display(fact.head(3))
display(dim.head(3))

## 2) Data Quality Snapshot

In [ ]:
# ==== DQ Snapshot ====
try:
    rr = json.load(open(RUN_RESULTS))
    statuses = [r.get("status") for r in rr.get("results",[])]
    summary = pd.Series(statuses).value_counts().to_dict()
    display({"DQ summary": summary})
except Exception as e:
    print("DQ summary not available:", e)

## 3) KPIs (Key Performance Indicators)

In [ ]:
# ==== KPIs ====
orders = fact["transaction_id"].nunique() if "transaction_id" in fact.columns else fact["cart_id"].nunique()
revenue = float(fact["total_sales"].sum())
items   = int(fact["quantity"].sum())
unique_products = fact["product_id"].nunique()
aov = revenue / orders if orders else 0.0

kpi_df = pd.DataFrame([{
    "Revenue": revenue,
    "Orders": orders,
    "Items": items,
    "Unique Products": unique_products,
    "AOV": aov
}])
display(kpi_df)

## 4) Visualizations

### 4.1) Daily Sales Trend

In [ ]:
# ==== Daily Sales Trend ====
daily = fact.groupby(fact["transaction_date"].dt.date)["total_sales"].sum()
plt.figure(figsize=(8,4))
daily.plot(kind="line")
plt.title("Daily Sales")
plt.xlabel("Date")
plt.ylabel("Revenue")
plt.tight_layout()
plt.show()

### 4.2) Top Categories by Revenue

In [ ]:
# ==== Top Categories ====
cat = fact.groupby("category")["total_sales"].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(8,4))
cat.plot(kind="bar")
plt.title("Top Categories by Revenue")
plt.xlabel("Category")
plt.ylabel("Revenue")
plt.tight_layout()
plt.show()

### 4.3) Top Products by Revenue

In [ ]:
# ==== Top Products ====
prod = fact.groupby(["product_id","product_name"])["total_sales"].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(8,4))
prod.plot(kind="bar")
plt.title("Top Products by Revenue")
plt.xlabel("Product")
plt.ylabel("Revenue")
plt.tight_layout()
plt.show()

### 4.4) Product Price Distribution (DIM)

In [ ]:
# ==== Price Distribution ====
plt.figure(figsize=(8,4))
dim["price"].dropna().plot(kind="hist", bins=30)
plt.title("Product Price Distribution")
plt.xlabel("Price")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

## 5) Insights (Ringkasan Temuan)

In [ ]:
# ==== Insights ====
# Hitung beberapa highlight untuk ringkasan naratif
import numpy as np
from IPython.display import display, Markdown

# Day with highest revenue
daily_rev = fact.groupby(fact["transaction_date"].dt.date)["total_sales"].sum()
top_day = daily_rev.idxmax() if not daily_rev.empty else None
top_day_val = float(daily_rev.max()) if not daily_rev.empty else None

# Top category by revenue
cat_rev = fact.groupby("category")["total_sales"].sum().sort_values(ascending=False)
top_cat = cat_rev.index[0] if not cat_rev.empty else None
top_cat_val = float(cat_rev.iloc[0]) if not cat_rev.empty else None

# Top product by revenue
prod_rev = fact.groupby(["product_id","product_name"])["total_sales"].sum().sort_values(ascending=False)
if not prod_rev.empty:
    (top_pid, top_pname), top_prod_val = prod_rev.index[0], float(prod_rev.iloc[0])
else:
    top_pid, top_pname, top_prod_val = None, None, None

INSIGHT_MD = f"""
**Ringkasan Temuan Utama**

- **Total Revenue**: {revenue:,.2f}
- **Total Orders**: {orders:,}
- **Total Items**: {items:,}
- **Unique Products**: {unique_products:,}
- **Average Order Value (AOV)**: {aov:,.2f}

Sorotan penjualan:
- **Kategori teratas** (berdasarkan revenue): {top_cat if top_cat else '-'} ({top_cat_val:,.2f} jika tersedia)
- **Produk teratas** (berdasarkan revenue): {top_pname if top_pname else '-'} (ID: {top_pid if top_pid else '-'}) {f'— {top_prod_val:,.2f}' if top_prod_val is not None else ''}
- **Hari performa terbaik**: {top_day if top_day else '-'} {f'— {top_day_val:,.2f}' if top_day_val is not None else ''}

Interpretasi singkat:
- Tren penjualan harian menunjukkan {('puncak yang jelas pada ' + str(top_day)) if top_day else 'pola yang relatif stabil'}.
- Komposisi revenue didorong oleh kategori **{top_cat if top_cat else 'N/A'}**; fokus promosi dan stok di kategori ini dapat meningkatkan performa.
- Produk **{top_pname if top_pname else 'N/A'}** berkontribusi signifikan; pertimbangkan strategi bundling atau cross-sell untuk memaksimalkan nilai pesanan.
"""

# Tampilkan sebagai Markdown di notebook
display(Markdown(INSIGHT_MD))

# Simpan juga sebagai file Markdown di direktori kerja notebook (opsional)
with open("analysis_summary.md", "w", encoding="utf-8") as f:
    f.write(INSIGHT_MD)
print("analysis_summary.md written")